Dependencies

In [ ]:
!pip install tensorflow
!pip install Keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.backend import set_session
import collections
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
import os
from keras.models import load_model
from keras.layers import GRU, Input, Dense, TimeDistributed
from keras.models import Model, Sequential
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import RepeatVector
from keras import losses
from keras.optimizers import SGD
from keras.layers import RepeatVector

Loading Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
f = open('/content/drive/MyDrive/Tamil.txt', 'r+', encoding="utf8")
x = f.readlines()

In [4]:
f = open('/content/drive/MyDrive/English.txt', 'r+', encoding="utf8")
y = f.readlines()

In [5]:
x = x[0:2000]
len(x)

2000

In [6]:
y = y[0:2000]

Text Preprocessing

In [7]:
x[0]= x[0].strip('\ufeffMMA')
y[0]= y[0].strip('\ufeffMMA')

In [8]:
import string
exclude = set(string.punctuation)
for i in range(0,len(x)):
    x[i] = x[i].strip('\n')
    x[i] = ''.join(ch for ch in x[i] if ch not in exclude)

In [9]:
for i in range(0,len(y)):
    y[i] = y[i].lower()
    y[i] = y[i].strip('\n')
    y[i] = ''.join(ch for ch in y[i] if ch not in exclude)

Cleaned text

In [10]:
print("Tamil Text:",x[1],"\n")
print("English Text:",y[1])

Tamil Text: சமீபகாலத்தில் சில தகவல்கள் யூலியஸ் ரோசன்பேர்க் ஒரு வித உளவுச்செய்தியை சோவியத் அதிகாரிகளுக்கு இரண்டாம் உலகப்போரின்போது அனுப்பியதில் சம்பந்தப்பட்டு இருந்ததாக வெளிவந்துள்ளன 

English Text: information has surfaced in recent years suggesting that julius rosenberg was involved in passing some form of intelligence to soviet officials during the second world war


In [11]:
len(x)

2000

In [12]:
len(y)

2000

In [13]:
english_words = []
for i in range(0,len(y)):
    english_words.append(y[i].split())   

In [14]:
english_words = [j for sub in english_words for j in sub]

In [15]:
print("Number of Unique English words:",len(set(english_words)))

Number of Unique English words: 8536


In [16]:
tamil_words = []
for i in range(0,len(x)):
    tamil_words.append(x[i].split())  
tamil_words = [j for sub in tamil_words for j in sub]

In [17]:
print("Number of Unique Tamil words:",len(set(tamil_words)))

Number of Unique Tamil words: 16010


In [18]:
tamilvocab = len(set(tamil_words))
engvocab = len(set(english_words))

In [19]:
length_tamil=[]
for i in range(0,len(x)):
    length_tamil.append(len(x[i].split()))

In [20]:
length_english=[]
for i in range(0,len(y)):
    length_english.append(len(y[i].split()))

In [21]:
sum(length_english)/len(length_english)

22.8965

In [22]:
sum(length_tamil)/len(length_tamil)

15.8825

In [23]:
print(max(length_english))
print(max(length_tamil))

112
76


In [24]:
english_words_counter = collections.Counter([word for sentence in y for word in sentence.split()])
tamil_words_counter = collections.Counter([word for sentence in x for word in sentence.split()])

In [25]:
english_words_counter.most_common(10)

[('the', 3786),
 ('of', 1747),
 ('and', 1620),
 ('to', 1253),
 ('in', 1052),
 ('a', 820),
 ('that', 587),
 ('for', 450),
 ('is', 448),
 ('on', 347)]

In [26]:
tamil_words_counter.most_common(10)

[('ஒரு', 319),
 ('மற்றும்', 316),
 ('என்று', 262),
 ('இந்த', 195),
 ('அவர்', 129),
 ('அமெரிக்க', 106),
 ('அரசியல்', 88),
 ('அவர்கள்', 86),
 ('நான்', 83),
 ('என்ற', 82)]

Tokenizer

In [ ]:
!pip install -U protobuf==3.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# config = tf.compat.v1.ConfigProto
# config.gpu_options.per_process_gpu_memory_fraction = 0.3
# set_session(tf.Session(config=config))

In [27]:
from keras.preprocessing.text import Tokenizer
def tokenize(x):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x) 
    return tokenizer.texts_to_sequences(x), tokenizer

In [28]:
x[0]

' கட்சியின் துணைத்தலைவர் க்வாஸி ஹுசேன் அகமத் சென்ற மாதம் பின்வருமாறு அறிவித்தார் நாங்கள் தீவிரவாதிகள் அல்ல'

In [29]:
z=(tokenize(x))
z[0][1]

[3767,
 53,
 747,
 3768,
 3769,
 1,
 3770,
 3771,
 326,
 1297,
 1298,
 3772,
 3773,
 3774,
 1975,
 3775]

In [30]:
z[1]

In [31]:
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',]
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'brown': 3, 'fox': 4, 'jumps': 5, 'over': 6, 'lazy': 7, 'dog': 8, 'by': 9, 'jove': 10, 'my': 11, 'study': 12, 'of': 13, 'lexicography': 14, 'won': 15, 'a': 16, 'prize': 17}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 3, 4, 5, 6, 1, 7, 8]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [9, 10, 11, 2, 12, 13, 14, 15, 16, 17]


Padding

In [32]:
def pad(x, length=None):
    return pad_sequences(x, maxlen=length, padding='post')

In [33]:
test_pad = pad(text_tokenized)
print("OUTPUT IS ALWAYS A LENGTH 10 ARRAY....FILLED BY 0s IN THE END")
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

OUTPUT IS ALWAYS A LENGTH 10 ARRAY....FILLED BY 0s IN THE END
Sequence 1 in x
  Input:  [1 2 3 4 5 6 1 7 8]
  Output: [1 2 3 4 5 6 1 7 8 0]
Sequence 2 in x
  Input:  [ 9 10 11  2 12 13 14 15 16 17]
  Output: [ 9 10 11  2 12 13 14 15 16 17]


Applying all tested preprocessing functions to our corpus

In [34]:
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    print('shape before: ', preprocess_y.shape)
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    print('shape after: ', preprocess_y.shape)
        
    return preprocess_x, preprocess_y, x_tk, y_tk

In [35]:
preprocess_x, x_tk = tokenize(x)
preprocess_x = pad(preprocess_x)
preprocess_x.shape

(2000, 76)

In [36]:
preproc_tamil_sentences, preproc_english_sentences, tamil_tokenizer, english_tokenizer =\
    preprocess(x, y)

shape before:  (2000, 112)
shape after:  (2000, 112, 1)


In [37]:
list(tamil_tokenizer.word_index.items())[:5]

[('ஒரு', 1), ('மற்றும்', 2), ('என்று', 3), ('இந்த', 4), ('அவர்', 5)]

In [38]:
list(english_tokenizer.word_index.items())[:5]

[('the', 1), ('of', 2), ('and', 3), ('to', 4), ('in', 5)]

Logits to text

In [39]:
def logits_to_text(logits, tokenizer):

    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = ''

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [40]:
print("tamil_sentences shape: ", preproc_english_sentences.shape)
print("english_sentences  shape: ", preproc_tamil_sentences.shape)
print('output sequence length: ', preproc_english_sentences.shape[1])

tamil_sentences shape:  (2000, 112, 1)
english_sentences  shape:  (2000, 76)
output sequence length:  112


In [41]:
tmp_x = pad(preproc_tamil_sentences, preproc_english_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[-2], 1))

In [42]:
tmp_x.shape

(2000, 112, 1)

In [43]:
from sklearn.model_selection import train_test_split

In [44]:
x_train,x_test , y_train , y_test = train_test_split(
...     tmp_x, preproc_english_sentences, test_size=0.2, random_state=42)

In [ ]:
def embed_model(input_shape, output_sequence_length, tamil_vocab_size, english_vocab_size, learning_rate=0.1):
    model = Sequential()
    model.add(Embedding(max(tamil_vocab_size, english_vocab_size) ,128 , input_length=output_sequence_length))
    model.add(LSTM(128, dropout=0.1, return_sequences=True))
    model.add(Dense(english_vocab_size, activation='softmax'))
    model.summary()
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

In [ ]:
#reshaping the input for processing embeddings
tmp_x = pad(preproc_tamil_sentences, preproc_english_sentences.shape[1])

In [ ]:
tmp_x.shape

(2000, 112)

In [ ]:
x_train,x_test , y_train , y_test = train_test_split(
...     tmp_x, preproc_english_sentences, test_size=0.2, random_state=42)

In [ ]:
embed_rnn_model = embed_model(
    x_train.shape,
    y_train.shape[1],
    len(tamil_tokenizer.word_index)+1,
    len(english_tokenizer.word_index)+1)
if os.path.exists(os.path.join("cache", "LSTM.h5"))== False:
    embedrnn = embed_rnn_model.fit(x_train, y_train, batch_size=100, epochs=50, validation_split=0.2)
else:
    embed_rnn_model = load_model(os.path.join("cache", "LSTM.h5"))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 112, 128)          2048768   
                                                                 
 lstm_1 (LSTM)               (None, 112, 128)          131584    
                                                                 
 dense_1 (Dense)             (None, 112, 8537)         1101273   
                                                                 
Total params: 3,281,625
Trainable params: 3,281,625
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
13/13 [==============================] - 65s 5s/step - loss: 4.2445 - accuracy: 0.7324 - val_loss: 3.3475 - val_accuracy: 0.7668
Epoch 2/50
13/13 [==============================] - 51s 4s/step - loss: 2.3395 - accuracy: 0.7915 - val_loss: 2.2627 - val_accuracy: 0.7979
Epoch 3/50
13/13 [==========

In [ ]:
embed_rnn_model.save(os.path.join("model", "LSTM.h5"))

In [ ]:
train_score = embed_rnn_model.evaluate(x_train, y_train, verbose=0)
print(" Train Accuracy: ", train_score[1])

 Train Accuracy:  0.8515178561210632


In [ ]:
test_score = embed_rnn_model.evaluate(x_test, y_test, verbose=0)
print(" Test Accuracy: ", test_score[1])

 Test Accuracy:  0.7681919932365417


In [ ]:
from nltk.translate.bleu_score import corpus_bleu

In [ ]:
x[15]

'வெளிநாட்டுக்கு ஷூட்டிங் கிளம்பும்வரை நன்றாகத்தான் இருந்தார் ஆபிரஹாம்'

In [ ]:
print(logits_to_text(embed_rnn_model.predict(tmp_x[:16])[15], english_tokenizer))

1/1 [==============================] - 1s 727ms/step
before the voyage overseas lobbying shooting society perfectly perfectly perfectly alright                                                                                                     


In [ ]:
x[10]

'வியட்நாம் தொடர்பாக அவர்கள் பொய் சொன்னார்கள் இப்போது மீண்டும் அதையே செய்து கொண்டிருக்கிறார்கள் என்று பேங்கரில் உள்ள ஒரு ஓய்வு பெற்ற அஞ்சல்துறை ஊழியர் கூறினார்'

In [ ]:
print(logits_to_text(embed_rnn_model.predict(tmp_x[:100])[10], english_tokenizer))

4/4 [==============================] - 1s 234ms/step
they lied as and that the you they hindi doing a of year working the that retired postal were bangor bangor the the the the the the the the the the the                                                                                


In [ ]:
(tmp_x[:100][10]).shape

(112,)

In [ ]:
tamil_tokenizer.word_index['அரசியல்']

7

In [ ]:
a = []
t = [0] * 74
s = [0] * 74

In [ ]:
t[0] = 10
a.append(t)
a.append(s)

In [ ]:
a=np.array(a)

In [ ]:
a[:1][0]

array([10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0])

In [ ]:
embed_rnn_model = embed_model(
    x_train.shape,
    y_train.shape[1],
    len(tamil_tokenizer.word_index)+1,
    len(english_tokenizer.word_index)+1)
if os.path.exists(os.path.join("cache", "LSTM2.h5"))== False:
    embedrnn = embed_rnn_model.fit(x_train, y_train, batch_size=150, epochs=70, validation_split=0.2)
else:
    embed_rnn_model = load_model(os.path.join("cache", "LSTM2.h5"))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 112, 128)          2048768   
                                                                 
 lstm_2 (LSTM)               (None, 112, 128)          131584    
                                                                 
 dense_2 (Dense)             (None, 112, 8537)         1101273   
                                                                 
Total params: 3,281,625
Trainable params: 3,281,625
Non-trainable params: 0
_________________________________________________________________
Epoch 1/70
9/9 [==============================] - 61s 7s/step - loss: 4.5436 - accuracy: 0.7011 - val_loss: 4.3867 - val_accuracy: 0.7920
Epoch 2/70
9/9 [==============================] - 52s 6s/step - loss: 3.4526 - accuracy: 0.7917 - val_loss: 3.5162 - val_accuracy: 0.7949
Epoch 3/70
9/9 [================

In [ ]:
embed_rnn_model.save(os.path.join("model", "LSTM2.h5"))

In [ ]:
train_score = embed_rnn_model.evaluate(x_train, y_train, verbose=0)
print(" Train Accuracy: ", train_score[1])

 Train Accuracy:  0.8597432971000671


In [ ]:
test_score = embed_rnn_model.evaluate(x_test, y_test, verbose=0)
print(" Test Accuracy: ", test_score[1])

 Test Accuracy:  0.7984821200370789


In [45]:
def embed_model2(input_shape, output_sequence_length, tamil_vocab_size, english_vocab_size, learning_rate=0.1):
    model = Sequential()
    model.add(Embedding(max(tamil_vocab_size, english_vocab_size) ,128 , input_length=output_sequence_length))
    model.add(LSTM(128, dropout=0.2, return_sequences=True))
    model.add(Dense(english_vocab_size, activation='softmax'))
    model.summary()
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

In [46]:
embed_rnn_model = embed_model2(
    x_train.shape,
    y_train.shape[1],
    len(tamil_tokenizer.word_index)+1,
    len(english_tokenizer.word_index)+1)
if os.path.exists(os.path.join("cache", "LSTM3.h5"))== False:
    embedrnn = embed_rnn_model.fit(x_train, y_train, batch_size=150, epochs=70, validation_split=0.2)
else:
    embed_rnn_model = load_model(os.path.join("cache", "LSTM3.h5"))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 112, 128)          2048768   
                                                                 
 lstm (LSTM)                 (None, 112, 128)          131584    
                                                                 
 dense (Dense)               (None, 112, 8537)         1101273   
                                                                 
Total params: 3,281,625
Trainable params: 3,281,625
Non-trainable params: 0
_________________________________________________________________
Epoch 1/70
9/9 [==============================] - 53s 5s/step - loss: 4.7354 - accuracy: 0.6957 - val_loss: 4.0053 - val_accuracy: 0.7969
Epoch 2/70
9/9 [==============================] - 48s 5s/step - loss: 3.2231 - accuracy: 0.7935 - val_loss: 3.0423 - val_accuracy: 0.7962
Epoch 3/70
9/9 [==================

In [47]:
embed_rnn_model.save(os.path.join("model", "LSTM3.h5"))

In [48]:
train_score = embed_rnn_model.evaluate(x_train, y_train, verbose=0)
print(" Train Accuracy: ", train_score[1])

 Train Accuracy:  0.8674163222312927


In [49]:
test_score = embed_rnn_model.evaluate(x_test, y_test, verbose=0)
print(" Test Accuracy: ", test_score[1])

 Test Accuracy:  0.780647337436676


In [50]:
def embed_model3(input_shape, output_sequence_length, tamil_vocab_size, english_vocab_size, learning_rate=0.1):
    model = Sequential()
    model.add(Embedding(max(tamil_vocab_size, english_vocab_size) ,128 , input_length=output_sequence_length))
    model.add(LSTM(128, dropout=0.2, return_sequences=True))
    model.add(Dense(english_vocab_size, activation='sigmoid'))
    model.summary()
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

In [51]:
embed_rnn_model = embed_model3(
    x_train.shape,
    y_train.shape[1],
    len(tamil_tokenizer.word_index)+1,
    len(english_tokenizer.word_index)+1)
if os.path.exists(os.path.join("cache", "LSTM4.h5"))== False:
    embedrnn = embed_rnn_model.fit(x_train, y_train, batch_size=150, epochs=70, validation_split=0.2)
else:
    embed_rnn_model = load_model(os.path.join("cache", "LSTM4.h5"))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 112, 128)          2048768   
                                                                 
 lstm_1 (LSTM)               (None, 112, 128)          131584    
                                                                 
 dense_1 (Dense)             (None, 112, 8537)         1101273   
                                                                 
Total params: 3,281,625
Trainable params: 3,281,625
Non-trainable params: 0
_________________________________________________________________
Epoch 1/70
9/9 [==============================] - 45s 5s/step - loss: 4.8133 - accuracy: 0.7008 - val_loss: 4.5991 - val_accuracy: 0.7986
Epoch 2/70
9/9 [==============================] - 42s 5s/step - loss: 3.8224 - accuracy: 0.7930 - val_loss: 3.9539 - val_accuracy: 0.7964
Epoch 3/70
9/9 [================

In [52]:
embed_rnn_model.save(os.path.join("model", "LSTM4.h5"))

In [53]:
train_score = embed_rnn_model.evaluate(x_train, y_train, verbose=0)
print(" Train Accuracy: ", train_score[1])

 Train Accuracy:  0.8353962302207947


In [54]:
test_score = embed_rnn_model.evaluate(x_test, y_test, verbose=0)
print(" Test Accuracy: ", test_score[1])

 Test Accuracy:  0.7931919693946838
